In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

In [2]:
data_path='../data/raw/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
full_data=pd.read_csv(data_path)

In [3]:
full_data.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1402,1402,1402,1406,1409,1409,1409,1409,1412,1415
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,275,280,284,290,296,301,306,312,316,319
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1491,1501,1510,1518,1523,1529,1539,1549,1556,1562
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,53,53,53,53,53,53,53,53,53,53
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,107,107,108,109,112,113,115,117,117,120


In [4]:
pd_data_base=full_data.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})
pd_data_base['state']=pd_data_base['state'].fillna('no')  

In [5]:
pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
pd_data_base.head()

,state,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20
0,no,Afghanistan,0,0,0,0,0,0,0,0,...,1402,1402,1402,1406,1409,1409,1409,1409,1412,1415
1,no,Albania,0,0,0,0,0,0,0,0,...,275,280,284,290,296,301,306,312,316,319
2,no,Algeria,0,0,0,0,0,0,0,0,...,1491,1501,1510,1518,1523,1529,1539,1549,1556,1562
3,no,Andorra,0,0,0,0,0,0,0,0,...,53,53,53,53,53,53,53,53,53,53
4,no,Angola,0,0,0,0,0,0,0,0,...,107,107,108,109,112,113,115,117,117,120


In [6]:
pd_relational=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
pd_relational.head()

,date,state,country,confirmed
0,1/22/20,Alberta,Canada,0.0
1,1/22/20,Anguilla,United Kingdom,0.0
2,1/22/20,Anhui,China,0.0
3,1/22/20,Aruba,Netherlands,0.0
4,1/22/20,Australian Capital Territory,Australia,0.0


In [7]:
pd_relational.dtypes

date          object
state         object
country       object
confirmed    float64
dtype: object

In [8]:
pd_relational['date']=pd_relational.date.astype('datetime64[ns]')

In [9]:
pd_relational.dtypes

date         datetime64[ns]
state                object
country              object
confirmed           float64
dtype: object

In [10]:
pd_relational.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)

In [12]:
JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',';',parse_dates=[0])
JH_data=JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",0.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


**test data**

In [13]:
test_data=JH_data[((JH_data['country']=='US')|
                      (JH_data['country']=='Germany'))&
                     (JH_data['date']>'2020-03-20')]


In [14]:
test_data.head()

,date,state,country,confirmed
15737,2020-03-21,no,Germany,84.0
15777,2020-03-21,no,US,463.0
16003,2020-03-22,no,Germany,94.0
16043,2020-03-22,no,US,573.0
16269,2020-03-23,no,Germany,123.0


In [15]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-09-07,no,9331.0
US,2020-09-07,no,189208.0


In [16]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [17]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-09-07     9331.0
      US      2020-09-07   189208.0

In [18]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result
    

In [19]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15737           NaN
                16003           NaN
                16269      5.145299
                16535      3.957672
                16801      3.903614
                            ...    
       US       59931    174.717466
                60200    183.542998
                60463    214.753242
                60732    317.725689
                60998    563.867662
Name: confirmed, Length: 342, dtype: float64

In [21]:
pd_DR_result=JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [22]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,465,NaN
2,Alberta,Canada,701,NaN
3,Alberta,Canada,966,NaN
4,Alberta,Canada,1263,NaN


In [24]:
JH_data=JH_data.reset_index()
JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",0.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


In [26]:
pd_result_larg=pd.merge(JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",0.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


**FILTERING WITH GROUPBY APPLY()**

In [27]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [29]:
pd_filtered_result=JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [30]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,"Korea, South",0.0,NaN,0.0
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0


**FILTERED DOUBLING RATE**

In [33]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
61175,no,Zimbabwe,59944,145.857143
61176,no,Zimbabwe,60180,293.333333
61177,no,Zimbabwe,60476,294.380952
61178,no,Zimbabwe,60712,258.500000
61179,no,Zimbabwe,61179,259.500000


In [34]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
61175,61175,2020-09-07,no,Barbados,7.0,inf,7.0,3.358979e+31
61176,61176,2020-09-07,no,Belarus,716.0,129.212121,715.8,1.421600e+02
61177,61177,2020-09-07,no,Belgium,9909.0,6604.888889,9909.6,3.810385e+03
61178,61178,2020-09-07,no,Albania,319.0,90.190476,320.0,6.856522e+01
61179,61179,2020-09-07,no,Zimbabwe,210.0,103.666667,208.4,2.595000e+02


In [35]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [36]:
pd_result_larg[pd_result_larg['country']=='Germany'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
59891,59891,2020-09-03,no,Germany,9322.0,1242.266667,9321.4,1757.761006
60159,60159,2020-09-04,no,Germany,9327.0,3729.466667,9326.0,1902.285714
60423,60423,2020-09-05,no,Germany,9329.0,2664.571429,9327.8,2914.083333
60691,60691,2020-09-06,no,Germany,9330.0,6219.111111,9329.9,4783.538462
60956,60956,2020-09-07,no,Germany,9331.0,9330.000000,9332.0,4442.809524


In [37]:
pd_result_larg.to_csv('../data/processed/COVID_final_set.csv',sep=';',index=False)